In [1]:
%cd ../

D:\Projects\EpiTB\tbdx23


In [2]:
import os
import pandas as pd
import numpy as np
from sim.core.cascade import RepoCascade
from sim.core.inputs import load_inputs
from sim.ind.dy import ModelIND, get_intv

In [3]:
iso = 'IND'
in_folder = f'results/B_{iso}'
out_folder = f'results/I_{iso}'

In [4]:
os.makedirs(out_folder, exist_ok=True)

In [5]:
pars_cs = pd.read_csv(f'results/pars_{iso}.csv')
repo_cs = RepoCascade(pars_cs, 2010)

inp = load_inputs(f'data/pars/{iso}')
model = ModelIND(inp)

In [6]:
particles = pd.read_csv(f'{in_folder}/Post.csv')
particles = [dict(row) for _, row in particles.iterrows()]

particles = particles[:10]

In [12]:
baseline = list()
for i, p in enumerate(particles):
    p['cas'] = repo_cs.sample()
    ys, ms, _ = model.simulate_to_fit(p, t_eval=np.linspace(2015, 2022, 8))
    baseline.append((p, ys, ms.assign(Key = i, Scenario = 'Baseline')))

In [14]:
scs = {
    'Pub90': lambda p: get_intv(p, pdx0_pub=0.9, pdx1_pub=0.9),
    'PubEng90': lambda p: get_intv(p, pdx0_pub=0.9, pdx1_pub=0.9, pdx0_eng=0.9, pdx1_eng=0.9),
    'RedDelay2': lambda p: get_intv(p, rd_csi=2, rd_recsi=2),
    'ACF2': lambda p: get_intv(p, rr_csi=2, rr_recsi=2)
}

In [15]:
intvs = list()

for i, (p, ys, _) in enumerate(baseline):
    _, ms0, _ = model.simulate_intv(p, ys)
    intvs.append(ms0.assign(Key = i, Scenario = 'Baseline'))
    
    for k, sc in scs.items():
        intv = sc(p)
        _, ms1, _ = model.simulate_intv(p, ys, intv)
        intvs.append(ms0.assign(Key = i, Scenario = k))

intvs = pd.concat(intvs)

In [17]:
mss0 = [ms for (_, _, ms) in baseline]
mss0 = pd.concat(mss0)
mss0.to_csv(f'{out_folder}/RunBaseline.csv')

In [18]:
intvs.to_csv(f'{out_folder}/RunIntv.csv')